<a href="https://colab.research.google.com/github/williamteles/Regression_Data_Mining/blob/main/Regression_Minera%C3%A7%C3%A3o_de_Dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%time
# instalação de dependências
try:
  from pycaret.regression import * 
except ImportError:
  print('Instalando dependências.')
  print('A EXECUÇÃO SERÁ PARADA! Por favor, execute novamente depois de parada.\n')
  # !pip install pycaret &> /dev/null
  !pip install -q pycaret
  !pip install -q tune-sklearn ray[tune]
  !pip install -q scikit-optimize
  import os
  os.kill(os.getpid(), 9)

CPU times: user 1.44 s, sys: 367 ms, total: 1.81 s
Wall time: 2.79 s


In [ ]:
import pandas as pd
import numpy as np

%matplotlib inline

In [ ]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00618/Steel_industry_data.csv"
df = pd.read_csv(url, delimiter=',')
df.head()

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,01/01/2018 00:45,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,01/01/2018 01:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,01/01/2018 01:15,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [ ]:
df.drop(df.columns[0], axis=1, inplace=True)

In [ ]:
df.shape

(35040, 10)

In [ ]:
df.isnull().sum()

Usage_kWh                               0
Lagging_Current_Reactive.Power_kVarh    0
Leading_Current_Reactive_Power_kVarh    0
CO2(tCO2)                               0
Lagging_Current_Power_Factor            0
Leading_Current_Power_Factor            0
NSM                                     0
WeekStatus                              0
Day_of_week                             0
Load_Type                               0
dtype: int64

In [ ]:
df.dtypes

Usage_kWh                               float64
Lagging_Current_Reactive.Power_kVarh    float64
Leading_Current_Reactive_Power_kVarh    float64
CO2(tCO2)                               float64
Lagging_Current_Power_Factor            float64
Leading_Current_Power_Factor            float64
NSM                                       int64
WeekStatus                               object
Day_of_week                              object
Load_Type                                object
dtype: object

In [ ]:
def replace_categories(data, cols):

  for col in cols:
    ordered_labels = df[col].unique()

    ordinal_label = {k: i for i, k in enumerate(ordered_labels)}
    print(col, ordinal_label)
    print()

    # use the dictionary to replace the categorical strings by integers
    data[col] = data[col].map(ordinal_label)

In [ ]:
categ_columns = df.select_dtypes(include=['object']).columns
replace_categories(df, categ_columns)

WeekStatus {'Weekday': 0, 'Weekend': 1}

Day_of_week {'Monday': 0, 'Tuesday': 1, 'Wednesday': 2, 'Thursday': 3, 'Friday': 4, 'Saturday': 5, 'Sunday': 6}

Load_Type {'Light_Load': 0, 'Medium_Load': 1, 'Maximum_Load': 2}



In [ ]:
df.duplicated().sum()

81

In [ ]:
df.eq(0).all(axis=1).any()

False

In [ ]:
df.describe()

,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,27.386892,13.035384,3.870949,0.011524,80.578056,84.367870,42750.000000,0.284932,2.991781,0.691781
std,33.444380,16.306000,7.424463,0.016151,18.921322,30.456535,24940.534317,0.451388,2.003433,0.792658
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000,0.000000,63.320000,99.700000,21375.000000,0.000000,1.000000,0.000000
50%,4.570000,5.000000,0.000000,0.000000,87.960000,100.000000,42750.000000,0.000000,3.000000,0.000000
75%,51.237500,22.640000,2.090000,0.020000,99.022500,100.000000,64125.000000,1.000000,5.000000,1.000000
max,157.180000,96.910000,27.760000,0.070000,100.000000,100.000000,85500.000000,1.000000,6.000000,2.000000


In [ ]:
num_columns = set(df.columns) - set(categ_columns)

num_columns.discard('Usage_kWh')

num_columns


{'CO2(tCO2)',
 'Lagging_Current_Power_Factor',
 'Lagging_Current_Reactive.Power_kVarh',
 'Leading_Current_Power_Factor',
 'Leading_Current_Reactive_Power_kVarh',
 'NSM'}

In [ ]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def limitar_anomalias(data, anomalia_cols):
  for col in anomalia_cols:
    q25, q75 = np.percentile(data[col], 25), np.percentile(data[col], 75)

    iiq = q75 - q25

    print(f'{bcolors.BOLD}Percentis da coluna {bcolors.ENDC}', end='')
    print(f'{bcolors.OKCYAN}{col}{bcolors.ENDC}: ', end='')
    print(f'25% = {bcolors.WARNING}{q25:.3f}{bcolors.ENDC}, ', end='')
    print(f'75% = {bcolors.WARNING}{q75:.3f}{bcolors.ENDC}, ', end='')
    print(f'IIQ = {bcolors.WARNING}{iiq:.3f}{bcolors.ENDC}')
    corte = iiq * 1.5

    inferior, superior = q25 - corte, q75 + corte

    anomalias = [x for x in data[col] if x < inferior or x > superior]

    print(f'{bcolors.BOLD}Outliers Identificados{bcolors.ENDC}: ', end='')
    print(f'{bcolors.WARNING}{len(anomalias)}{bcolors.ENDC}')

    print(bcolors.OKGREEN + '#' * 100 + bcolors.ENDC + '\n')

    data[col] = np.clip(data[col], inferior, superior)
  
  return data

In [ ]:
df2 = limitar_anomalias(df.copy(), num_columns)

Percentis da coluna Leading_Current_Reactive_Power_kVarh: 25% = 0.000, 75% = 2.090, IIQ = 2.090
Outliers Identificados: 7759
####################################################################################################

Percentis da coluna Lagging_Current_Power_Factor: 25% = 63.320, 75% = 99.023, IIQ = 35.703
Outliers Identificados: 1
####################################################################################################

Percentis da coluna NSM: 25% = 21375.000, 75% = 64125.000, IIQ = 42750.000
Outliers Identificados: 0
####################################################################################################

Percentis da coluna CO2(tCO2): 25% = 0.000, 75% = 0.020, IIQ = 0.020
Outliers Identificados: 437
####################################################################################################

Percentis da coluna Lagging_Current_Reactive.Power_kVarh: 25% = 2.300, 75% = 22.640, IIQ = 20.340
Outliers Identificados: 1059
#########################

In [ ]:
df2.describe()

,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,27.386892,12.790640,1.304613,0.011392,80.578335,99.811411,42750.000000,0.284932,2.991781,0.691781
std,33.444380,15.577588,2.197324,0.015785,18.920207,0.319341,24940.534317,0.451388,2.003433,0.792658
min,0.000000,0.000000,0.000000,0.000000,9.766250,99.250000,0.000000,0.000000,0.000000,0.000000
25%,3.200000,2.300000,0.000000,0.000000,63.320000,99.700000,21375.000000,0.000000,1.000000,0.000000
50%,4.570000,5.000000,0.000000,0.000000,87.960000,100.000000,42750.000000,0.000000,3.000000,0.000000
75%,51.237500,22.640000,2.090000,0.020000,99.022500,100.000000,64125.000000,1.000000,5.000000,1.000000
max,157.180000,53.150000,5.225000,0.050000,100.000000,100.000000,85500.000000,1.000000,6.000000,2.000000


In [ ]:
reg = setup(data=df2, target='Usage_kWh', silent=True, verbose=False,
            fold_shuffle=True, fold=30)

In [ ]:
best = compare_models(sort='RMSE')

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,0.7023,4.3318,2.0602,0.9961,0.0909,0.0472,6.7183
et,Extra Trees Regressor,0.7191,4.6402,2.1368,0.9958,0.0927,0.0475,5.3273
lightgbm,Light Gradient Boosting Machine,1.0439,5.1339,2.2477,0.9954,0.1035,0.0765,0.2160
dt,Decision Tree Regressor,0.9424,7.3940,2.6886,0.9934,0.1018,0.0541,0.1200
gbr,Gradient Boosting Regressor,1.9200,10.6044,3.2480,0.9905,0.1417,0.1208,1.8923
knn,K Neighbors Regressor,2.2755,25.4257,5.0331,0.9771,0.1827,0.1141,0.1050
lr,Linear Regression,2.7822,28.2635,5.2453,0.9748,0.1812,0.1600,0.1163
br,Bayesian Ridge,2.7827,28.2637,5.2454,0.9748,0.1814,0.1602,0.0297
lar,Least Angle Regression,2.7826,28.2700,5.2459,0.9748,0.1812,0.1599,0.0207
omp,Orthogonal Matching Pursuit,2.9538,32.3793,5.5916,0.9711,0.2067,0.1604,0.0183


In [ ]:
model = create_model('lightgbm')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,1.0843,4.9595,2.2270,0.9954,0.1098,0.0783
1,1.0435,5.1609,2.2718,0.9952,0.1113,0.0794
2,1.0438,8.5265,2.9200,0.9922,0.0967,0.0718
3,1.1438,5.8077,2.4099,0.9949,0.1067,0.0789
4,1.0721,4.7593,2.1816,0.9960,0.0992,0.0752
5,1.0004,3.7797,1.9442,0.9966,0.0972,0.0787
6,1.1147,6.6368,2.5762,0.9940,0.1020,0.0743
7,1.0538,5.4658,2.3379,0.9952,0.0969,0.0754
8,1.0890,4.7230,2.1732,0.9958,0.1067,0.0795
9,1.0067,3.5935,1.8957,0.9970,0.1048,0.0817


In [ ]:
tuned_model = tune_model(model, n_iter=50, optimize='RMSE',
                         search_library='tune-sklearn',
                         search_algorithm='bayesian',
                         early_stopping=True, 
                         early_stopping_max_iters=10)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.8025,4.0994,2.0247,0.9962,0.0929,0.0576
1,0.7583,2.5687,1.6027,0.9976,0.0966,0.0609
2,0.7765,4.6883,2.1652,0.9957,0.0792,0.0511
3,0.8613,4.9688,2.2291,0.9956,0.0905,0.0583
4,0.8106,4.5501,2.1331,0.9962,0.0839,0.0535
5,0.7494,2.8647,1.6925,0.9974,0.0807,0.0552
6,0.8081,4.6380,2.1536,0.9958,0.0891,0.0539
7,0.8154,4.5359,2.1298,0.9960,0.0908,0.0558
8,0.7564,3.3407,1.8278,0.9970,0.0843,0.0543
9,0.7408,3.1450,1.7734,0.9974,0.0907,0.0576


In [ ]:
evaluate_model(tuned_model)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [ ]:
type(tuned_model)

lightgbm.sklearn.LGBMRegressor

In [ ]:
finalized_model = finalize_model(tuned_model)

In [ ]:
predict_model(finalized_model)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Light Gradient Boosting Machine,0.599,2.0708,1.439,0.9982,0.0833,0.0512


,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus_1,Day_of_week_0,Day_of_week_1,Day_of_week_2,Day_of_week_3,Day_of_week_4,Day_of_week_5,Day_of_week_6,Load_Type_0,Load_Type_1,Load_Type_2,Usage_kWh,Label
0,27.860001,0.000,0.03,92.779999,100.000000,53100.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,69.300003,70.026101
1,3.670000,0.000,0.00,57.209999,100.000000,85500.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.560000,2.563078
2,0.000000,5.225,0.00,100.000000,99.250000,73800.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4.360000,5.291264
3,2.300000,0.040,0.00,78.139999,99.989998,2700.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.880000,3.072315
4,25.920000,0.000,0.03,90.410004,100.000000,51300.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,54.830002,54.671503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10508,26.389999,0.000,0.02,88.919998,100.000000,72900.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,51.299999,51.390953
10509,4.030000,0.000,0.00,57.200001,100.000000,11700.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.810000,2.837427
10510,23.620001,0.000,0.03,92.820000,100.000000,69300.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,58.900002,58.208627
10511,19.660000,0.000,0.02,93.440002,100.000000,35100.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,51.549999,51.427768
